In [1]:
import shutil
import time

import os
import SimpleITK as sitk
import pydicom as pydicom

from tqdm import tqdm

import subprocess

import pandas as pd

from concurrent.futures import ThreadPoolExecutor
NUM_WORKERS = 16

%matplotlib notebook
import gui
import registration_gui as rgui

In [2]:
DATA_DIR = r"/home/thulasiseetha/research/dataset/raw/BrainMR"

# Generating Brain & Tissue Segmentation Masks For T1vibe FS Images

In [3]:
pids = [fname.split(".")[0] for fname in os.listdir(os.path.join(DATA_DIR, "t1v_fs", "imgs"))]

In [4]:
def execute_command(command):

    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    output, error = process.communicate()
    return (command, output, error)

def process(pid, sequence):
    
    IMG_DIR = os.path.join(DATA_DIR, sequence, "imgs")
    OUT_MASK_DIR = os.path.join(DATA_DIR, sequence, "masks")
    
    img_file = os.path.join(IMG_DIR, f"{pid}.nii.gz")
    out_mask_dir = os.path.join(OUT_MASK_DIR, pid)
    
    if not os.path.exists(out_mask_dir):
        os.makedirs(out_mask_dir)
        
    brainMask_file = os.path.join(out_mask_dir, "brainMask.nii.gz")
    brainMaskedImg_file = os.path.join(out_mask_dir, "brainMaskedImg.nii.gz")
    
    command = f"mri_synthstrip -i {img_file} -m {brainMask_file} -o {brainMaskedImg_file}"
    
    command, output, error = execute_command(command)
    
    if not error:
        
        t = 1 if "t1" in sequence else 2 #otherwise its t2
        n = 3 if "t1" in sequence else 4 #otherwise its t2
        
        #Also, if you are segmenting T2-weighted images, you may need to select 4 classes so that dark non-brain matter is processed correctly (this is not a problem with T1-weighted as CSF and dark non-brain matter look similar).
    
        tissueMask_prefix = os.path.join(out_mask_dir, "tissue")
        command = f"fast -t {t} -o {tissueMask_prefix} -n {n} {brainMaskedImg_file}"

        command, output, error = execute_command(command)
        
    if error:
        print(f"Error executing command '{command}': {error.decode()} for {pid}")
    
    pbar.update()
    

In [8]:
pbar = tqdm(pids, desc = "Processing patients", position=0)

with ThreadPoolExecutor(max_workers=NUM_WORKERS) as e:e.map(process, pids, ["t2w_nfs"]*len(pids))

Processing patients: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 52/52 [24:05<00:00, 34.78s/it]

In [9]:
pbar = tqdm(pids, desc = "Processing patients", position=0)

with ThreadPoolExecutor(max_workers=NUM_WORKERS) as e:e.map(process, pids, ["t2w_fs"]*len(pids))

Processing patients: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 52/52 [13:07<00:00, 12.22s/it]

# Registering T2w_nfs with T2w_fs and exporting them along with masks

In [10]:
OUT_DATA_DIR = r"/home/thulasiseetha/research/dataset/curated/BrainMR"

In [11]:
def register_using_rigid(fixed_image, moving_image, verbose=False):
    
    RIGID_NUM_ITER = 100
    
    initial_transform = sitk.CenteredTransformInitializer(
    fixed_image,
    moving_image,
    sitk.Euler3DTransform(),
    sitk.CenteredTransformInitializerFilter.GEOMETRY,
    )
    
    registration_method = sitk.ImageRegistrationMethod()

    # Similarity metric settings.
    registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
    registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
    registration_method.SetMetricSamplingPercentage(0.01)

    registration_method.SetInterpolator(sitk.sitkLinear)

    # Optimizer settings.
    registration_method.SetOptimizerAsGradientDescent(learningRate=1.0,numberOfIterations=RIGID_NUM_ITER,convergenceMinimumValue=1e-6,convergenceWindowSize=10,)
    registration_method.SetOptimizerScalesFromPhysicalShift()

    # Don't optimize in-place, we would possibly like to run this cell multiple times.
    registration_method.SetInitialTransform(initial_transform, inPlace=False)
    
    if verbose:
        # Connect all of the observers so that we can perform plotting during registration.
        registration_method.AddCommand(sitk.sitkStartEvent, rgui.start_plot)
        registration_method.AddCommand(sitk.sitkEndEvent, rgui.end_plot)
        registration_method.AddCommand(sitk.sitkMultiResolutionIterationEvent, rgui.update_multires_iterations)
        registration_method.AddCommand(sitk.sitkIterationEvent, lambda: rgui.plot_values(registration_method))

    final_transform = registration_method.Execute(fixed_image, moving_image)

    if verbose:
        # Always check the reason optimization terminated.
        print("Final metric value: {0}".format(registration_method.GetMetricValue()))
        print("Optimizer's stopping condition, {0}".format(registration_method.GetOptimizerStopConditionDescription()))
        
    
    return final_transform

In [12]:
def register(pid, verbose=False):
    
    t2w_fs_img = fixed_img = sitk.ReadImage(PAT_RECORDS[pid]["t2w_fs"]["img"], sitk.sitkFloat32)
    t2w_nfs_img = moving_img = sitk.ReadImage(PAT_RECORDS[pid]["t2w_nfs"]["img"], sitk.sitkFloat32)
    
    t2w_nfs_mask = sitk.ReadImage(PAT_RECORDS[pid]["t2w_nfs"]["mask"])
    
    final_transform = register_using_rigid(fixed_img, moving_img, verbose)

    reg_t2w_nfs_img = sitk.Resample(
        t2w_nfs_img,
        t2w_fs_img,
        final_transform,
        sitk.sitkLinear,
        0.0,
        moving_img.GetPixelID(),
        useNearestNeighborExtrapolator=True
    )
    
    t2w_fs_mask = reg_t2w_nfs_mask = sitk.Resample(
        t2w_nfs_mask,
        t2w_fs_img,
        final_transform,
        sitk.sitkNearestNeighbor,
        0.0,
        moving_img.GetPixelID(),
    )
    
    out_dir = os.path.join(OUT_DATA_DIR, "t2w_nfs", pid)
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
        
    sitk.WriteImage(sitk.Cast(reg_t2w_nfs_img, sitk.sitkUInt16), os.path.join(out_dir, "img.nii.gz"))
    sitk.WriteImage(sitk.Cast(reg_t2w_nfs_mask, sitk.sitkUInt16), os.path.join(out_dir, "mask.nii.gz"))
    
    
    out_dir = os.path.join(OUT_DATA_DIR, "t2w_fs", pid)
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
        
    sitk.WriteImage(sitk.Cast(t2w_fs_img, sitk.sitkUInt16), os.path.join(out_dir, "img.nii.gz"))
    sitk.WriteImage(sitk.Cast(t2w_fs_mask, sitk.sitkUInt16), os.path.join(out_dir, "mask.nii.gz"))
    
    
    
    pbar.update()
    
    
    

In [13]:
PAT_RECORDS = {}

for pid in tqdm(pids):
    
    PAT_RECORDS[pid] = {"t2w_nfs":{"img":os.path.join(DATA_DIR, "t2w_nfs", "imgs", f"{pid}.nii.gz" ), "mask":os.path.join(DATA_DIR, "t2w_nfs", "masks", pid, "tissue_seg.nii.gz")},
                        "t2w_fs":{"img":os.path.join(DATA_DIR, "t2w_fs", "imgs", f"{pid}.nii.gz")}}

pbar = tqdm(pids, desc="registration and curation in progress", position=0)
with ThreadPoolExecutor(max_workers=NUM_WORKERS) as e:e.map(register, pids)
    


registration and curation in progress: 100%|██████████████████████████████████████████████████████████████████████████| 52/52 [00:37<00:00,  5.29it/s]